# Libraries

Import libraries needed in this project.

In [2]:
import warnings
warnings.filterwarnings('ignore')
import cv2  #For image and video processing and visualization
import os   #To interact with operating system and files
import numpy as np  #For matrix operations
import random   #To generate random numbers
from sklearn.mixture import GaussianMixture  #For clustering
from sklearn.cluster import DBSCAN   #For clustering
from sklearn.cluster import MeanShift, estimate_bandwidth  #For clustering
import numba
from PIL import Image

# Open Video

This function recieves a video path and returns a capture stream.

In [3]:
def open_vid(input_file): #Video path
    cap = cv2.VideoCapture(input_file) #Open capture stream
    if not cap.isOpened(): #Check if is available
        print("Error: Could not open video.")
    return cap

# Get Video Properties

This function gets the video's properties for its width and height in pixels, frames per second (fps) and frame count.

In [4]:
def get_props(cap, display=1): #Video capture stream and flag to display properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) #Get Width
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) #Get Height
    fps = cap.get(cv2.CAP_PROP_FPS) #Get FPS
    count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) #Get Frame Count
    if display==1:  #If flag is 1, display properties
        print("Width: ",width)
        print("Height: ",height)
        print("FPS: ",fps)
        print("Frame Count: ",count)
    return width,height,fps,count

# Get Frames from Video as a List 

This function takes the capture stream of a video and saves its frames in a list.

In [5]:
def get_frames(cap): #Video capture stream
    frames = [] #Frames list
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) #Get Frame count
    for i in range(frame_count): #For each frame
        ret, frame = cap.read() # Read a frame from the video
        if not ret: #If couldn't read frame
            print("Error: Could not read frame.") #Display error message and return read frames
            return frames
        frames.append(frame) # Save the frame to the list
    if not frames: #If list is empty
        print("No frames were saved.") #Display error message 
    return frames

# Delete PNG Files in Directory

This function deletes all PNG image files in the specified directory path.

In [6]:
def delete_png_files(directory_path): #Directory path to delete all PNG files
    for filename in os.listdir(directory_path): # List all files in the specified directory
        file_path = os.path.join(directory_path, filename) # Construct full file path
        try:
            if os.path.isfile(file_path) and filename.lower().endswith('.png'): # Check if it's a PNG file and remove it
                os.remove(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')  #If not, display message

# Save Frames as Images

This function saves a list of frames as images in the given directory path.

In [7]:
def save_frames(frames, frame_folder): #Frame list, directory path to be saved
    if not os.path.exists(frame_folder):  # Create the folder to save frames if it doesn't exist
        os.makedirs(frame_folder)
    frame_count = 0 #frame index
    for i in range(len(frames)): #for each frame
        frame_filename = os.path.join(frame_folder, f'frame_{frame_count:03d}.png') #directory path and file name
        cv2.imwrite(frame_filename, frames[i]) # Save the frame as an image file
        frame_count += 1 #Next frame index

# Read Images in a Directory

This function reads and stores images from a given directory path to a list.

In [8]:
def read_images(directory_path):
    images = []
    # List all files in the directory
    for filename in os.listdir(directory_path):
        # Check if the file has a PNG extension
        if filename.lower().endswith('.png'):
            # Construct full file path
            file_path = os.path.join(directory_path, filename)
            # Read the image using OpenCV
            image = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)  # cv2.IMREAD_UNCHANGED to keep the alpha channel if present
            if image is not None:
                images.append(image)
            else:
                print(f"Failed to read image: {file_path}")
    return images

# Create Video with List of Frames

This function creates and saves the frames in a list into a video file in the specified directory path.

In [9]:
def save_vid(frames,output_file,fps): #List of frames, directory path to be saved, fps
    height, width, _ = frames[0].shape     # Get frame dimensions
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Define the codec and create VideoWriter object
    out = cv2.VideoWriter(output_file, fourcc, fps, (width, height)) 
    [out.write(frame) for frame in frames]; # Write the frames to the new video file
    out.release() # Release the video writer object

# Display Image

This function displays a given image in a window.

In [10]:
def display_frame(I): #Image to display
    cv2.imshow('I', I)  #Display Image
    cv2.waitKey(0) #Press any key to stop displaying
    cv2.destroyAllWindows() #Close all windows

This function display a list of images at the same time.

In [11]:
def display_images(images):
    for i, img in enumerate(images):
        window_name = f'Image {i+1}'
        cv2.imshow(window_name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Play Frames in a Window

This function displays a video made out of a list of frames with the specified FPS.

In [12]:
def play_frames(frames,fps): #list of frames, fps
    delay = int(1000/fps) #Delay between frames
    print("Delay: ",delay)
    for frame in frames:
        cv2.imshow('Video Playback', frame) # Display the frame
        if cv2.waitKey(delay) & 0xFF == ord('q'): # Exit the playback if 'q' is pressed
            break
    cv2.destroyAllWindows() #Close all windows

# Optical Flow

Optical Flow can be seen as a vector field that describes the movement between two consecutive images or frames in a video. There are many ways to calculate the oprical flow. Some of the methods to solve optical flow are:
- Ferneback
- Lucas - Kanade
- Phase Correlation

## Farneback Magnitude

This function take a list of frames to return a list of the optical flow's vector fields and display the magnitude of these vectors.

In [13]:
def OFM(frames): #List of frames
    OF = []  #List of optical flow's vector field
    prev_gray = cv2.cvtColor(frames[0], cv2.COLOR_BGR2GRAY) #Convert to Grayscale
    i = 0  #frame's index
    while True:
        print(i,end='\r') #Print frame index
        next_gray=cv2.cvtColor(frames[(i+1)%len(frames)],cv2.COLOR_BGR2GRAY)#Convert current & next frame to grayscale
        # Calculate the dense optical flow using Farneback method
        flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, 0.5, 3, 10, 10, 5, 1.2, 0)
                                            # prev     next      flow  dist lvl win it smooth std  flag
        # Visualize the optical flow
        hsv = np.zeros_like(frames[i]) #Matrix with shape like frames with zeros
        if len(hsv.shape) != 3 or hsv.shape[2] != 3: #If color image
            hsv = np.zeros((frames[i].shape[0], frames[i].shape[1], 3)) #Matrix of size of frame with 3 color channels
        hsv[..., 1] = 255 # ch1 Saturation (Full)
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1]) # Cartesian to Polar
        hsv[..., 0] = ang * 180 / np.pi / 2  #Angle
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX) #Normalize from 0 to 255
        flow_rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)   #Convert to RGB
        OF.append(flow) #Add to list of Optical Flow
        flow_M  = flow[...,0] + flow[...,1] #Add components of vectors
        flow_M = flow_M/np.max(flow_M)*255 #Calculate Magnitude of vectors     
        # Display the original frames and the optical flow magnitudes
        cv2.imshow('Prev Frame', frames[i])
        cv2.imshow('Next Frame', frames[(i+1) % len(frames)])
        cv2.imshow('Optical Flow', flow_rgb)
        cv2.imshow('Optical Flow Mag', flow_M)
        # Wait for a key press to move to the next frame
        key = cv2.waitKeyEx(0)
        if key == ord('q'):
            break
        if key == 2424832:  # Left arrow key to move backwards
            i = i - 1
            if i<0:
                i = 0
        if key == 2555904:  # Right arrow key to move forwards
            i = i + 1
            if i>(len(frames)-1):
                i = len(frames)-1
        prev_gray = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY) # Update the previous frame and grayscale image
    cv2.destroyAllWindows() # Release the video capture object and close all OpenCV windows
    return OF

## Draw Optical Flow Vector Field and Sum of all Vectors

These functions recieve the optical flow and the frame to draw onto.

In [14]:
def draw_optical_flow_vectors(flow, frame, step): #Optical flow, frame, window size
    h, w = frame.shape[:2] #frame size
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2, -1).astype(int) #Grid of window centers
    fx, fy = flow[y, x].T #Separate flow components
    mask = np.zeros_like(frame) # Create a mask to draw the vectors  
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2) # Create line endpoints
    lines = np.int32(lines + 0.5) # Add space between
    # Draw lines and circles for each vector
    for (x1, y1), (x2, y2) in lines:
        cv2.line(mask, (x2, y2), (x1, y1), (0, 255, 0), 1)
        cv2.circle(frame, (x2, y2), 1, (0, 255, 0), -1)
    return cv2.add(frame, mask) #Draw mask on top of frame

def draw_sum_vector(flow, frame):
    h, w = frame.shape[:2] #Frame size
    # Compute the sum of all flow vectors by components
    sum_fx = np.sum(flow[..., 0])
    sum_fy = np.sum(flow[..., 1])
    center_x, center_y = w // 2, h // 2 # Calculate the center point of the frame
    # Normalize the sum vector to fit within the image
    max_length = min(w, h) // 2
    vector_length = np.sqrt(sum_fx**2 + sum_fy**2)
    if vector_length > 0:
        scale = max_length / vector_length #scale factor
        end_x = int(center_x + sum_fx * scale)
        end_y = int(center_y + sum_fy * scale)
    else:
        end_x, end_y = center_x, center_y
    # Draw the sum vector as a red arrow
    frame_with_vector = np.copy(frame) #copy original frame
    cv2.arrowedLine(frame_with_vector, (center_x, center_y), (end_x, end_y), (0, 0, 255), 2, tipLength=0.2) 
    return frame_with_vector

## Farneack with Vectors

The function OFV take a list of frames to return and display the optical flow's vector fields as well as the sum of all vectors.

In [15]:
def OFV(frames, step=15): #Frame list, window size
    OF = [] #List of Optical flow's vector fields
    i = 0 #Frame's index
    while True:
        #convert it to grayscale
        prev_gray = cv2.cvtColor(np.copy(frames[i]), cv2.COLOR_BGR2GRAY)
        next_gray = cv2.cvtColor(np.copy(frames[i+1]), cv2.COLOR_BGR2GRAY)
        # Calculate the dense optical flow using Farneback method
        flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        # Draw the optical flow vectors on the frame
        flow_frame = draw_optical_flow_vectors(flow, cv2.addWeighted(frames[i], 0.5, frames[i+1], 0.5, 0), step)
        OF.append(flow) #Add flow to list
        # Draw the sum vector on the frame
        frame_with_sum_vector = draw_sum_vector(flow, flow_frame)
        # Display the original frame with optical flow vectors and sum vector
        cv2.imshow('Prev Frame', frames[i])
        cv2.imshow('Next Frame', frames[i+1])
        #cv2.imshow('Optical Flow Vectors', flow_frame)
        cv2.imshow('Sum Vector', frame_with_sum_vector)
        key = cv2.waitKeyEx(0) #Read pressed key
        if key == ord('q'): #Stop if pressed
            break
        if key == 2424832:  # Left arrow key for previous frames
            i = i - 1
            if i < 0: #prevent non existent frames
                i = 0
        if key == 2555904:  # Right arrow key for next frames
            i = i + 1
            if i > (len(frames) - 2): #prevent non existent frames
                i = len(frames) - 2
    cv2.destroyAllWindows() #Close all windows
    return OF

## Lucas-Kanade

This function utilizes the Lukas-Kanade method to solve and display the optical flow.

In [16]:
def OFLK(frames, step_size=15): #Video Frames, window size
    OF = [] #List of optical flow
    lk_params=dict(winSize=(step_size,step_size),maxLevel=10,criteria=(cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_COUNT,30,0.01))
    #lucas-kanade parameters  #window Size        #PyramidLevel   #End Criteria
    i = 0 #Frame index
    while i < len(frames) - 1:
        #Convert to Grayscale
        old_gray = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY) 
        frame_gray = cv2.cvtColor(frames[i + 1], cv2.COLOR_BGR2GRAY)
        #Neighborhood grid centers
        grid_y, grid_x = np.mgrid[step_size//2:old_gray.shape[0]:step_size, step_size//2:old_gray.shape[1]:step_size]
        p0 = np.vstack((grid_x.ravel(), grid_y.ravel())).T.astype(np.float32).reshape(-1, 1, 2)
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params) #Calculate Optical Flow
        #Set to 1 if change has been found
        good_new = p1[st == 1]
        good_old = p0[st == 1]
        #Frames average
        frame = cv2.addWeighted(frames[i], 0.5, frames[i + 1], 0.5, 0)
        #Calculate Vector Field
        flow = np.zeros((old_gray.shape[0], old_gray.shape[1], 2))
        flow[good_old[:, 1].astype(int), good_old[:, 0].astype(int), 0] = good_new[:, 0] - good_old[:, 0]
        flow[good_old[:, 1].astype(int), good_old[:, 0].astype(int), 1] = good_new[:, 1] - good_old[:, 1]
        #Draw Vectors
        frame_with_vectors = draw_optical_flow_vectors(flow, frame, step_size)
        frame_with_sum_vector = draw_sum_vector(flow, frame_with_vectors)
        OF.append(flow) #add to flow list
        #Display Frames and Flow
        cv2.imshow('Previous Frame',frames[i])
        cv2.imshow('Next Frame', frames[i+1])
        #cv2.imshow('Optical Flow Vectors', frame_with_vectors)
        cv2.imshow('Sum Vector', frame_with_sum_vector)
        key = cv2.waitKeyEx(0)
        if key == ord('q'):
            break
        if key == 2424832:  # Left arrow key to move backward
            i = max(0, i - 1)
        if key == 2555904:  # Right arrow key to move forward
            i = min(len(frames) - 2, i + 1)
    cv2.destroyAllWindows()
    return OF

## Phase Correlation

The PhaseC function calculates and displays the optical flow of the given frames by using Phase Correlation.

In [17]:
def draw_sum_vector_phase_c(sum_dx, sum_dy, frame): #Y component, X component, frame to drawn on
    h, w = frame.shape[:2] #Size
    center_x, center_y = w // 2, h // 2 # Calculate the center point of the frame
    # Normalize the sum vector to fit within the image
    max_length = min(w, h) // 2
    vector_length = np.sqrt(sum_dx**2 + sum_dy**2)
    if vector_length > 0:
        scale = 0.1#max_length / vector_length #Scale factor
        end_x = int(center_x + sum_dx * scale)
        end_y = int(center_y + sum_dy * scale)
    else:
        end_x, end_y = center_x, center_y
    # Draw the sum vector as a red arrow
    frame_with_vector = np.copy(frame)
    cv2.arrowedLine(frame_with_vector, (center_x, center_y), (end_x, end_y), (0, 0, 255), 2, tipLength=0.2)
    return frame_with_vector

def PhaseC(frames, block_size=15, grid_step=15):
    OF = [] #Optical Flow List
    i = 0 #Frame index
    while True:
        prev_frame = np.copy(frames[i])
        next_frame = np.copy(frames[i + 1])

        # Convert frames to grayscale
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        next_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)

        # Create an image to visualize the flow
        flow_img = cv2.cvtColor(prev_gray, cv2.COLOR_GRAY2BGR)
        frame = cv2.addWeighted(frames[i], 0.5, frames[i+1], 0.5, 0)

        sum_dx, sum_dy = 0, 0

        # Iterate over the grid
        for y in range(0, prev_gray.shape[0] - block_size, grid_step):
            for x in range(0, prev_gray.shape[1] - block_size, grid_step):
                # Extract the blocks
                prev_block = prev_gray[y:y + block_size, x:x + block_size]
                next_block = next_gray[y:y + block_size, x:x + block_size]

                # Compute phase correlation
                shift, _ = cv2.phaseCorrelate(prev_block.astype(np.float32), next_block.astype(np.float32))
                dx, dy = shift

                # Sum the vectors
                sum_dx += dx
                sum_dy += dy

                # Scale down the length of the arrows and size of the tips
                scale = 1
                tip_length = 0.2

                # Draw the vector on the flow image
                cv2.arrowedLine(frame, (x + block_size // 2, y + block_size // 2),
                                (int(x + block_size // 2 + dx * scale), int(y + block_size // 2 + dy * scale)),
                                (0, 255, 0), 1, tipLength=tip_length)

        # Draw the sum vector on the frame
        frame_with_sum_vector = draw_sum_vector_phase_c(sum_dx, sum_dy, frame)

        OF.append(flow_img)

        # Display the frames and the flow
        cv2.imshow('Previous Frame', prev_frame)
        cv2.imshow('Next Frame', next_frame)
        #cv2.imshow('Optical Flow', frame)
        cv2.imshow('Sum Vector', frame_with_sum_vector)

        # Wait for a key press to move to the next frame
        key = cv2.waitKeyEx(0)
        if key == ord('q'):
            break
        if key == 2424832:  # Left arrow key
            i = i - 1
            if i < 0:
                i = 0
        if key == 2555904:  # Right arrow key
            i = i + 1
            if i > (len(frames) - 2):
                i = len(frames) - 2

    cv2.destroyAllWindows()
    return OF

# Real Time Optical Flow with Camera

This function captures video from a camera and displays it with the optical flow using Furneback's method.

In [18]:
def draw_optical_flow(frame, flow, step=16, max_magnitude=100):
    h, w = frame.shape[:2]
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    lines = np.vstack([x, y, x + fx, y + fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    vis = frame.copy()
    for (x1, y1), (x2, y2) in lines:
        cv2.arrowedLine(vis, (x1, y1), (x2, y2), (0, 255, 0), 1, tipLength=0.4)

    # Compute the sum of all flow vectors
    sum_fx = np.sum(fx)
    sum_fy = np.sum(fy)
    
    # Normalize the sum vector
    norm = np.sqrt(sum_fx**2 + sum_fy**2)
    if norm > 0:
        sum_fx /= norm
        sum_fy /= norm

    # Scale the normalized vector by the maximum magnitude
    sum_fx *= min(norm, max_magnitude)
    sum_fy *= min(norm, max_magnitude)
    
    # Draw the red vector representing the normalized sum of all flow vectors
    center_x, center_y = w // 2, h // 2
    end_x = int(center_x + sum_fx)  # Scale for better visualization
    end_y = int(center_y + sum_fy)
    cv2.arrowedLine(vis, (center_x, center_y), (end_x, end_y), (0, 0, 255), 2, tipLength=0.4)
    
    return vis

def cap_of():
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    ret, prev_frame = cap.read()
    if not ret:
        print("Error: Could not read initial frame.")
        return
    
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 
                                            0.5, 3, 15, 3, 5, 1.2, 0)
        
        prev_gray = gray

        # Draw the optical flow vectors on the RGB frame
        optical_flow_frame = draw_optical_flow(frame, flow)
        
        # Display the resulting frame with optical flow
        cv2.imshow('Webcam Video with Optical Flow', optical_flow_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Frames Differences

This function takes a list of frames of a video and calculates the absolute diffences between two consecutive frames with a threshold to display a binary image with the regions where the most differences are.

In [19]:
def frame_dif(frames, threshold=30): #List of frames, threshold
    D = [] #Differences List
    i = 0 #Frame index
    while True:
        # Convert frames to grayscale
        prev_gray = cv2.cvtColor(np.copy(frames[i]),cv2.COLOR_BGR2GRAY)
        curr_gray = cv2.cvtColor(np.copy(frames[i+1]), cv2.COLOR_BGR2GRAY)
        # Compute absolute difference between frames
        diff = cv2.absdiff(prev_gray, curr_gray) 
        # Create a mask to highlight pixels with significant changes
        mask = np.zeros_like(diff)
        mask[diff > threshold] = 255 #Apply Threshold
        D.append(mask) #Add to list
        # Show original frame and mask
        cv2.imshow('Prev Frame', frames[i])
        cv2.imshow('Next Frame', frames[i+1])
        cv2.imshow('Pixels with Most Changes', mask)
         # Wait for a key press to move to the next frame
        key = cv2.waitKeyEx(0)
        if key == ord('q'):
            break
        if key == 2424832:  # Left arrow key
            i = i - 1
            if i<0:
                i = 0
        if key == 2555904:  # Right arrow key
            i = i + 1
            if i>(len(frames)-2):
                i = len(frames)-2
        prev_frame = frames[i]
    # Release the video capture object
    cv2.destroyAllWindows()
    return D

This function takes a list of frames and calculates the normalized absolute difference between 2 consecutive frames.

In [20]:
def frame_dif1(frames): #List of frames, threshold
    D = [] #Differences List
    i = 0 #Frame index
    while True:
        # Convert frames to grayscale
        prev_gray = cv2.cvtColor(np.copy(frames[i]), cv2.COLOR_BGR2GRAY)
        curr_gray = cv2.cvtColor(np.copy(frames[i+1]), cv2.COLOR_BGR2GRAY)
        # Compute absolute difference between frames
        diff = abs(prev_gray-curr_gray) #Absolute difference
        dif = (1/diff.max())*diff*255 #Normalize
        D.append(diff) #Add to list
        # Show original frame and mask
        cv2.imshow('Prev Frame', frames[i])
        cv2.imshow('Next Frame', frames[i+1])
        cv2.imshow('Pixels with Most Changes', diff)
         # Wait for a key press to move to the next frame
        key = cv2.waitKeyEx(0)
        if key == ord('q'):
            break
        if key == 2424832:  # Left arrow key
            i = i - 1
            if i<0:
                i = 0
        if key == 2555904:  # Right arrow key
            i = i + 1
            if i>(len(frames)-2):
                i = len(frames)-2
        prev_frame = frames[i]
    # Release the video capture object
    cv2.destroyAllWindows()
    return D

# Alter Frames

The next fucntions are used to alter the frames in a video.

## Change Color Channel's Ranges

This function changes the range of each individual color channel.

In [21]:
def change_range_colors(image, min_vals=(0, 0, 0), max_vals=(255, 255, 255)):
    # Split the image into its BGR channels
    b, g, r = cv2.split(image)
    # Clip each channel to its respective range
    b = np.clip(b, min_vals[0], max_vals[0])
    g = np.clip(g, min_vals[1], max_vals[1])
    r = np.clip(r, min_vals[2], max_vals[2])
    # Merge the channels back together
    new_image = cv2.merge((b, g, r))
    return new_image

## Add Occlusions

This function adds occlusions in the shape of rectangles and/or circles into the image.

In [22]:
def occlusions(image, num_occlusions=1,loc=[],sizes=[],shapes=['rectangle','circle'], colors=(-1,-1,-1)):    
    output_image = np.copy(image) #Copy of frame
    height, width = image.shape[:2] #Gets size
    num_occlusions = num_occlusions if len(loc)==0 else len(loc) #number of occlusions
    # Draw occlusions on the image
    for i in range(num_occlusions):
        shape_type = random.choice(shapes)
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) if colors==(-1,-1,-1) else colors  # Random color
        if shape_type == 'rectangle':
            x = random.randint(0, width - 1) if len(loc)==0 else loc[i][0]
            y = random.randint(0, height - 1) if len(loc)==0 else loc[i][1]
            width_rect = random.randint(5, width//2) if len(sizes)==0 else loc[i][0]
            height_rect = random.randint(5, height//2) if len(sizes)==0 else loc[i][1]
            cv2.rectangle(output_image, (x, y), (x + width_rect, y + height_rect), color, -1)  # Filled rectangle
        elif shape_type == 'circle':
            center = (random.randint(0, width - 1), random.randint(0, height - 1)) if len(loc)==0 else (loc[i][0],loc[i][1])
            radius = 50#random.randint(5, 100)
            cv2.circle(output_image, center, radius, color, -1)  # Filled circle
    return output_image

## Draw Random Lines

This functions draws straight lines in an image.

In [23]:
def rnd_lines(image, num_lines):
    # Get the dimensions of the image
    height, width = image.shape[:2]
    # Copy the image to avoid modifying the original
    output_image = image.copy()
    for _ in range(num_lines):
        # Generate random start point
        start_point = (random.randint(0, width-1), random.randint(0, height-1))
        # Generate a random angle and length for the line
        angle = random.uniform(0, 2 * np.pi)
        length = random.randint(1, min(width, height) // 2)  # Limit length to half of the smallest dimension
        # Calculate the end point using the angle and length
        end_point = (int(start_point[0] + length * np.cos(angle)), 
                     int(start_point[1] + length * np.sin(angle)))
        # Ensure the end point is within the image boundaries
        end_point = (min(max(end_point[0], 0), width-1), min(max(end_point[1], 0), height-1))
        # Generate a random color (BGR format)
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        # Generate a random thickness for the line
        thickness = random.randint(1, 10)
        # Draw the line on the image
        cv2.line(output_image, start_point, end_point, color, thickness)
    return output_image

## Random Region's Color Change

This function changes the color channel's range in a certain number of rectangle or ellipse regions.

In [24]:
def rnd_regions(image, num_regions):
    # Get the dimensions of the image
    height, width = image.shape[:2]
    # Copy the image to avoid modifying the original
    output_image = image.copy()
    for _ in range(num_regions):
        # Generate random region shape and size
        region_shape = random.choice(['rectangle', 'ellipse'])
        if region_shape == 'rectangle':
            region_width = random.randint(10, width // 3)
            region_height = random.randint(10, height // 3)
            top_left_x = random.randint(0, width - region_width)
            top_left_y = random.randint(0, height - region_height)
            # Define the region
            region = output_image[top_left_y:top_left_y + region_height, top_left_x:top_left_x + region_width]
        elif region_shape == 'ellipse':
            center_x = random.randint(width // 3, width - width // 3)
            center_y = random.randint(height // 3, height - height // 3)
            axis_length = (random.randint(10, width // 3), random.randint(10, height // 3))
            angle = random.randint(0, 360)
            start_angle = 0
            end_angle = 360
            # Create a mask for the ellipse
            mask = np.zeros((height, width), dtype=np.uint8)
            cv2.ellipse(mask, (center_x, center_y), axis_length, angle, start_angle, end_angle, 255, -1)
            # Extract the region using the mask
            region = cv2.bitwise_and(output_image, output_image, mask=mask)
        # Change color channels within the region
        for channel in range(3):  # Assuming BGR format
            # Generate random ranges for the color channel
            low = random.randint(0, 255)
            high = random.randint(low, 255)
            if region_shape == 'rectangle':
                region[..., channel] = np.clip(region[..., channel], low, high)
            elif region_shape == 'ellipse':
                # Apply changes to the region using the mask
                channel_region = output_image[..., channel]
                channel_region[mask == 255] = np.clip(channel_region[mask == 255], low, high)
                output_image[..., channel] = channel_region
        if region_shape == 'rectangle':
            # Place the modified region back into the image for rectangles
            output_image[top_left_y:top_left_y + region_height, top_left_x:top_left_x + region_width] = region
    return output_image

# Simple Image Cartoonization

The next functions are used to cartoonize an image pixel wise by using different algorithms and tecniques.

In [25]:
def cartoonize_image(img, k=10, it = 10, t1 = 150, t2 = 255, ks = 1 , kc=1):
    # Apply bilateral filter to smooth the image
    img_color = cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)
    # Convert to grayscale
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply Gaussian Blur
    img_blur = cv2.GaussianBlur(img_gray, (kc, kc), 0)
    # Detect edges using Canny edge detection
    edges = cv2.Canny(img_blur, threshold1=t1, threshold2=t2)
    # Dilate the edges to make them more prominent
    kernel = np.ones((ks, ks), np.uint8)
    edges = cv2.dilate(edges, kernel, iterations=1)
    # Invert the edges
    edges = cv2.bitwise_not(edges)
    # Convert edges back to color, so we can combine with color image
    edges_colored = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
    # Perform K-means clustering
    img_data = np.float32(img_color).reshape((-1, 3))
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, it, 0.2)
    _, labels, centers = cv2.kmeans(img_data, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    centers = np.uint8(centers)
    img_clustered = centers[labels.flatten()]
    img_clustered = img_clustered.reshape(img_color.shape)
    # Combine edge and clustered image
    cartoon = cv2.bitwise_and(img_clustered, edges_colored)
    return cartoon

# Image Variability

The following functions calculate different variability indexes of an image.

In [26]:
def relative_luminance(image):
    b, g, r = cv2.split(image)
    luminance = 0.2126 * r + 0.7152 * g + 0.0722 * b
    #return np.mean(luminance)
    return luminance.sum()/(image.shape[0]*image.shape[1]*255)

def color_variance(image):
    variance_b = np.var(image[:, :, 0])
    variance_g = np.var(image[:, :, 1])
    variance_r = np.var(image[:, :, 2])
    return ((variance_b+variance_g+variance_r)/3)/(image.shape[0]*image.shape[1])

def calculate_variability(image,kernel_size=3):
    kernel = np.ones((kernel_size,kernel_size))/(kernel_size**2-1)
    kernel[kernel_size//2,kernel_size//2] = -1
    V = cv2.filter2D(image,-1,kernel)
    return abs(V).sum()/image.size

# Color Limitation

In [78]:
def color_limit(img,N_colors=20):
    # Convert BGR (OpenCV format) to RGB (PIL format)
    image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Convert NumPy array (RGB) to PIL Image
    pil_image = Image.fromarray(image_rgb)
    # Apply the quantize method in PIL
    quantized_image = pil_image.quantize(N_colors)
    # Convert the quantized PIL image back to NumPy array (RGB format)
    image_rgb_back = np.array(quantized_image.convert('RGB'))  # Convert quantized image back to RGB
    #Convert RGB back to BGR for displaying with OpenCV
    image_bgr_back = cv2.cvtColor(image_rgb_back, cv2.COLOR_RGB2BGR)
    return image_bgr_back

# Pallete Applier

In [91]:
palette = [
    (0, 0, 0),   # Black
    (255, 255, 255),  # White
    (255, 0, 0),   # Red
    (0, 255, 0),   # Green
    (0, 0, 255),   # Blue
    # Add more colors as needed
]

def apply_palette(img, palette):
    img = img.convert("RGB")
    palette_img = Image.new("P", (1, 1))
    palette_img.putpalette(sum(palette, ()))
    return cv2.cvtColor(np.array(img.quantize(palette=palette_img).convert("RGB")),cv2.COLOR_BGR2RGB)

# Add One Inconsistency to Frame

In [ ]:
def AddOneInc(F):
    Bin = np.zeros(F[0].shape[:2],dtype=np.uint8)
    I = F.copy()
    x,y = random.randint(0,2*I.shape[0]//3),random.randint(0,2*I.shape[1]//3)
    l = random.randint(20,I.shape[1]//5)
    Op = random.randint(0,3)
    if Op==0:
        I[x:x+l,y:y+l] = change_range_colors(I[x:x+l,y:y+l],(random.randint(30,140),random.randint(30,140),random.randint(30,140)),(random.randint(150,255),random.randint(150,255),random.randint(150,255)))
        Bin[x:x+l,y:y+l]=255
    elif Op==1:
        R,G,B = random.randint(0,255),random.randint(0,255),random.randint(0,255)
        I[x:x+l,y:y+l] = change_range_colors(I[x:x+l,y:y+l],(R,G,B),(R,G,B))
        Bin[x:x+l,y:y+l]=255
    elif Op==2:
        Thick = random.randint(1,10)
        l2 = random.randint(10,I.shape[1]//8)
        I = cv2.line(I,(x,y),(x+l,y+l2),(random.randint(0,255),random.randint(0,255),random.randint(0,255)),Thick)
        Bin = cv2.line(Bin,(x,y),(x+l,y+l),255,Thick)
    else:
        Thick = random.randint(1,10)
        l2 = random.randint(10,I.shape[1]//8)
        I = cv2.line(I,(x,y),(x+l,y+l2),(0,0,0),Thick)
        Bin = cv2.line(Bin,(x,y),(x+l,y+l),255,Thick)
    return I,Bin

# Add Inconsistencies to Video Each N Frames

In [37]:
def AddInc(F,N=3):
    Inc = []
    Bin = [np.zeros(F[0].shape[:2],dtype=np.uint8) for _ in F]
    for i in range(len(F)):
        if i%N==0 and i!=0:
            I = F[i].copy()
            x,y = random.randint(0,2*I.shape[0]//3),random.randint(0,2*I.shape[1]//3)
            print(I.shape)
            l = random.randint(20,I.shape[1]//5)
            Op = random.randint(0,3)
            if Op==0:
                I[x:x+l,y:y+l] = change_range_colors(I[x:x+l,y:y+l],(random.randint(30,140),random.randint(30,140),random.randint(30,140)),(random.randint(150,255),random.randint(150,255),random.randint(150,255)))
                Bin[i][x:x+l,y:y+l]=255
            elif Op==1:
                R,G,B = random.randint(0,255),random.randint(0,255),random.randint(0,255)
                I[x:x+l,y:y+l] = change_range_colors(I[x:x+l,y:y+l],(R,G,B),(R,G,B))
                Bin[i][x:x+l,y:y+l]=255
            elif Op==2:
                Thick = random.randint(1,10)
                l2 = random.randint(10,I.shape[1]//8)
                I = cv2.line(I,(x,y),(x+l,y+l2),(random.randint(0,255),random.randint(0,255),random.randint(0,255)),Thick)
                Bin[i] = cv2.line(Bin[i],(x,y),(x+l,y+l),255,Thick)
            else:
                Thick = random.randint(1,10)
                l2 = random.randint(10,I.shape[1]//8)
                I = cv2.line(I,(x,y),(x+l,y+l2),(0,0,0),Thick)
                Bin[i] = cv2.line(Bin[i],(x,y),(x+l,y+l),255,Thick)
            Inc.append(I)
        else:
            Inc.append(F[i])
    return Inc,Bin

# Cartoonize Video using K-Means

In [27]:
def cartoonize_vid(F,t1 = 150, t2 = 255, ks = 1 , kc=1):
    C = []
    centers = np.uint8(np.asarray([[r,g,b] for r in range(0,255,40) for g in range(0,255,40) for b in range(0,255,40)]))
    It = 0
    for f in F:
        It = It+1
        print(It,'/',len(F),end='\r')
        img_color = cv2.bilateralFilter(f, d=9, sigmaColor=75, sigmaSpace=75)
        # Convert to grayscale
        img_gray = cv2.cvtColor(f, cv2.COLOR_RGB2GRAY)
        # Apply Gaussian Blur
        img_blur = cv2.GaussianBlur(img_gray, (kc, kc), 0)
        # Detect edges using Canny edge detection
        edges = cv2.Canny(img_blur, threshold1=t1, threshold2=t2)
        # Dilate the edges to make them more prominent
        kernel = np.ones((ks, ks), np.uint8)
        edges = cv2.dilate(edges, kernel, iterations=1)
        # Invert the edges
        edges = cv2.bitwise_not(edges)
        # Convert edges back to color, so we can combine with color image
        edges_colored = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
        # Perform K-means clustering
        img_data = np.float32(img_color).reshape((-1, 3))

        distances = np.linalg.norm(img_data[:, np.newaxis] - centers, axis=2)
        closest_clusters = np.argmin(distances, axis=1)

        # Map the pixels in the second image to the closest cluster centers
        segmented_pixels2 = centers[closest_clusters]
        segmented_image2 = segmented_pixels2.reshape(f.shape)

        # Combine edge and clustered image
        cartoon = cv2.bitwise_and(segmented_image2, edges_colored)
        C.append(cartoon)
    return C

# Canny Edges

In [28]:
def canny_edge(image,tl=150,th=255,inverted=True):
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply Canny edge detector
    edges = cv2.Canny(gray_image, threshold1=tl, threshold2=th)
    # Invert the binary image (0 becomes 255, and 255 becomes 0)
    if inverted:
        edges = cv2.bitwise_not(edges)
    return edges.astype(np.uint8)

# Sobel Edges

In [29]:
def sobel_edge(image,tl=150,th=255):
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Compute the gradients using the Sobel operator
    sobelx = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)  # Gradient in x-direction
    sobely = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)  # Gradient in y-direction
    # Calculate the magnitude of the gradient
    magnitude = np.sqrt(sobelx**2 + sobely**2)
    # Normalize to range 0 to 255 and convert to uint8
    magnitude = np.uint8(255 * magnitude / np.max(magnitude))
    # Convert to binary image (0 or 255) and invert it
    _, binary_edges = cv2.threshold(magnitude, tl, th, cv2.THRESH_BINARY_INV)
    return binary_edges

# Prewitt Edges

In [30]:
def prewitt_edge(image,tl=150,th=255):
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Define Prewitt kernels
    kernelx = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]], dtype=int)
    kernely = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]], dtype=int)
    # Apply the Prewitt operator
    prewittx = cv2.filter2D(gray_image, -1, kernelx)
    prewitty = cv2.filter2D(gray_image, -1, kernely)
    # Calculate the magnitude of the gradient
    magnitude = np.sqrt(prewittx**2 + prewitty**2)
    # Normalize to range 0 to 255 and convert to uint8
    magnitude = np.uint8(255 * magnitude / np.max(magnitude))
    # Convert to binary image (0 or 255) and invert it
    _, binary_edges = cv2.threshold(magnitude, tl, th, cv2.THRESH_BINARY_INV)
    return binary_edges

# Scharr Edges

In [31]:
def scharr_edge(image,tl=150,th=255):
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Compute the gradients using the Scharr operator
    scharrx = cv2.Scharr(gray_image, cv2.CV_64F, 1, 0)  # Gradient in x-direction
    scharry = cv2.Scharr(gray_image, cv2.CV_64F, 0, 1)  # Gradient in y-direction
    # Calculate the magnitude of the gradient
    magnitude = np.sqrt(scharrx**2 + scharry**2)
    # Normalize to range 0 to 255 and convert to uint8
    magnitude = np.uint8(255 * magnitude / np.max(magnitude))
    # Convert to binary image (0 or 255) and invert it
    _, binary_edges = cv2.threshold(magnitude, tl, th, cv2.THRESH_BINARY_INV)
    return binary_edges

# Laplacian Edges

In [32]:
def laplacian_edge(image,tl=150,th=255):
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply Laplacian operator to find edges
    laplacian_edges = cv2.Laplacian(gray_image, cv2.CV_64F)
    # Convert to absolute values and normalize to range 0 to 255
    laplacian_edges = cv2.convertScaleAbs(laplacian_edges)
    # Convert to binary image (0 or 255) and invert it
    _, binary_edges = cv2.threshold(laplacian_edges, tl, th, cv2.THRESH_BINARY_INV)
    return binary_edges

# Read First N Frames from Videos in Directory

In [33]:
def read_first_frames(directory,N):
    video_frames = []  # Dictionary to store video names and their first 10 frames
    for filename in os.listdir(directory):
        if filename.endswith(".mp4"):  # Check for video file extensions
            video_path = os.path.join(directory, filename)
            cap = cv2.VideoCapture(video_path)
            frames = []
            frame_count = 0
            while frame_count < N and cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break  # If the video ends before 10 frames
                frames.append(frame)
                frame_count += 1
            cap.release()  # Release the video capture object
            video_frames += frames  # Store the frames with the video name
    return video_frames

# Calculate Gradients from 2 Images

In [93]:
def calc_grads(img1, img2):
    # Calculate the spatial gradients using Sobel operator
    gx1 = cv2.Sobel(img1, cv2.CV_64F, 1, 0, ksize=5)
    gy1 = cv2.Sobel(img1, cv2.CV_64F, 0, 1, ksize=5)
    gx2 = cv2.Sobel(img2, cv2.CV_64F, 1, 0, ksize=5)
    gy2 = cv2.Sobel(img2, cv2.CV_64F, 0, 1, ksize=5)
    # Calculate the temporal gradient
    tg = cv2.absdiff(img2, img1)
    # Normalize gradients for display purposes
    gx1d = cv2.convertScaleAbs(gx1)
    gy1d = cv2.convertScaleAbs(gy1)
    gx2d = cv2.convertScaleAbs(gx2)
    gy2d = cv2.convertScaleAbs(gy2)
    tgd = cv2.convertScaleAbs(tg)
    return gx1,gy1,gx2,gy2,tg,gx1d,gy1d,gx2d,gy2d,tgd

# Calculate Movement with Gradients

In [92]:
def calc_grad_movement(img1,img2):
    # Calculate gradients
    gx1,gy1,gx2,gy2,tg,gx1d,gy1d,gx2d,gy2d,tgd = calculate_and_display_gradients(img1, img2)
    # Calculate the magnitude and angle of optical flow
    magnitude = cv2.magnitude(gx2,gy2)
    angle = cv2.phase(gx2,gy2,angleInDegrees=True)
    # Normalize magnitude for display purposes
    magnitude_display = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    magnitude_display = cv2.convertScaleAbs(magnitude_display)
    # Optional: Visualize flow direction using HSV color space
    hsv = np.zeros((img1.shape[0], img1.shape[1], 3), dtype=np.uint8)
    hsv[..., 1] = 255
    hsv[..., 0] = angle / 2
    hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    optical_flow = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return magnitude_display,optical_flow

# Calculate Frames Differences with 3 Channels

In [95]:
def calc_differences(img1,img2):
    # Convert images to RGB (OpenCV loads images in BGR format by default)
    img1_rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    # Convert images to grayscale
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    # Calculate the absolute difference between the images
    diff_rgb = np.abs(img1_rgb - img2_rgb)
    diff_gray = np.abs(img1_gray.astype(np.float32) - img2_gray.astype(np.float32))
    # Normalize the differences to [0, 255]
    norm_diff_rgb = cv2.normalize(diff_rgb, None, 0, 255, cv2.NORM_MINMAX)
    norm_diff_gray = cv2.normalize(diff_gray, None, 0, 255, cv2.NORM_MINMAX)
    # Split the RGB difference into color channels
    diff_r, diff_g, diff_b = cv2.split(norm_diff_rgb)
    # Convert the single channel differences back to 3-channel images for display
    diff_r_colored = cv2.merge([diff_r, np.zeros_like(diff_r), np.zeros_like(diff_r)])
    diff_g_colored = cv2.merge([np.zeros_like(diff_g), diff_g, np.zeros_like(diff_g)])
    diff_b_colored = cv2.merge([np.zeros_like(diff_b), np.zeros_like(diff_b), diff_b])
    return diff_r_colored,diff_g_colored,diff_b_colored,norm_diff_gray.astype(np.uint8)

# Calculate Temporal Gradient for each Color Channel

In [97]:
def calc_temporal_gradient(img1, img2):
    # Convert images to RGB (OpenCV loads images in BGR format by default)
    img1_rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    # Convert images to grayscale
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    # Calculate the temporal gradient (difference) between the images
    grad_rgb = np.abs(img2_rgb - img1_rgb)
    grad_gray = np.abs(img2_gray.astype(np.float32) - img1_gray.astype(np.float32))
    # Normalize the gradients to [0, 255]
    norm_grad_rgb = cv2.normalize(grad_rgb, None, 0, 255, cv2.NORM_MINMAX)
    norm_grad_gray = cv2.normalize(grad_gray, None, 0, 255, cv2.NORM_MINMAX)
    # Split the RGB gradient into color channels
    grad_r, grad_g, grad_b = cv2.split(norm_grad_rgb)
    # Convert the single channel gradients back to 3-channel images for display
    grad_r_colored = cv2.merge([grad_r, np.zeros_like(grad_r), np.zeros_like(grad_r)])
    grad_g_colored = cv2.merge([np.zeros_like(grad_g), grad_g, np.zeros_like(grad_g)])
    grad_b_colored = cv2.merge([np.zeros_like(grad_b), np.zeros_like(grad_b), grad_b])
    return grad_r_colored,grad_g_colored,grad_b_colored,norm_grad_gray.astype(np.uint8)

# Temporal Consistency Index

In [ ]:
def calc_temporal_consistency(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        raise ValueError("Error opening video file. Check the file path.")
    
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_count < 2:
        raise ValueError("Video must have at least two frames to calculate temporal consistency.")
    
    # Initialize variables to accumulate error values
    total_error_rgb = np.zeros(3)  # R, G, B channels
    total_error_gray = 0
    num_frames = 0

    # Read the first frame
    ret, prev_frame = cap.read()
    if not ret:
        raise ValueError("Failed to read the first frame.")
    
    prev_frame_rgb = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2RGB)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    
    while True:
        # Read the next frame
        ret, curr_frame = cap.read()
        if not ret:
            break
        
        curr_frame_rgb = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2RGB)
        curr_frame_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
        
        # Compute the absolute difference between consecutive frames
        diff_rgb = np.abs(curr_frame_rgb.astype(np.float32) - prev_frame_rgb.astype(np.float32))
        diff_gray = np.abs(curr_frame_gray.astype(np.float32) - prev_frame_gray.astype(np.float32))
        
        # Calculate the mean absolute error for the current frame pair
        mean_error_rgb = np.mean(diff_rgb, axis=(0, 1))  # Mean error for R, G, B channels
        mean_error_gray = np.mean(diff_gray)
        
        total_error_rgb += mean_error_rgb
        total_error_gray += mean_error_gray
        num_frames += 1
        
        # Update previous frame
        prev_frame_rgb = curr_frame_rgb
        prev_frame_gray = curr_frame_gray
    
    # Compute the average temporal consistency index
    if num_frames == 0:
        raise ValueError("No frames processed. Check the video file.")
    
    temporal_consistency_index_rgb = total_error_rgb / num_frames
    temporal_consistency_index_gray = total_error_gray / num_frames
    
    # Release video capture object
    cap.release()
    
    return temporal_consistency_index_rgb, temporal_consistency_index_gray

In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.color import rgb2gray

def calculate_temporal_gradient(image1, image2):
    # Convert images to RGB and grayscale
    img1_rgb = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    img1_gray = rgb2gray(img1_rgb)
    img2_gray = rgb2gray(img2_rgb)

    # Calculate the temporal gradient (difference) for each channel and grayscale
    grad_rgb = np.abs(img2_rgb.astype(np.float32) - img1_rgb.astype(np.float32))
    grad_gray = np.abs(img2_gray.astype(np.float32) - img1_gray.astype(np.float32))

    return grad_rgb, grad_gray

def calculate_similarity_indexes(image1, image2):
    # Convert images to RGB and grayscale
    img1_rgb = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    img1_gray = rgb2gray(img1_rgb)
    img2_gray = rgb2gray(img2_rgb)
    
    # Resize images to the same size if they are different
    if img1_rgb.shape != img2_rgb.shape:
        img2_rgb = cv2.resize(img2_rgb, (img1_rgb.shape[1], img1_rgb.shape[0]))
        img2_gray = cv2.resize(img2_gray, (img1_gray.shape[1], img1_gray.shape[0]))

    # Compute Mean Squared Error (MSE) for RGB
    mse_rgb = np.mean((img1_rgb - img2_rgb) ** 2)
    
    # Compute Structural Similarity Index (SSIM) for grayscale
    ssim_gray, _ = ssim(img1_gray, img2_gray, full=True)
    
    # Compute Color Histograms for RGB
    hist1_r = cv2.calcHist([img1_rgb[:, :, 0]], [0], None, [256], [0, 256])
    hist2_r = cv2.calcHist([img2_rgb[:, :, 0]], [0], None, [256], [0, 256])
    hist1_g = cv2.calcHist([img1_rgb[:, :, 1]], [0], None, [256], [0, 256])
    hist2_g = cv2.calcHist([img2_rgb[:, :, 1]], [0], None, [256], [0, 256])
    hist1_b = cv2.calcHist([img1_rgb[:, :, 2]], [0], None, [256], [0, 256])
    hist2_b = cv2.calcHist([img2_rgb[:, :, 2]], [0], None, [256], [0, 256])
    
    hist_corr_r = cv2.compareHist(hist1_r, hist2_r, cv2.HISTCMP_CORREL)
    hist_corr_g = cv2.compareHist(hist1_g, hist2_g, cv2.HISTCMP_CORREL)
    hist_corr_b = cv2.compareHist(hist1_b, hist2_b, cv2.HISTCMP_CORREL)
    
    # Compute Edge-based similarity (using Canny edge detector)
    edges1 = cv2.Canny(cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY), 100, 200)
    edges2 = cv2.Canny(cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY), 100, 200)
    
    edge_similarity = np.sum(edges1 == edges2) / np.size(edges1)
    
    return {
        'MSE_RGB': mse_rgb,
        'SSIM_Gray': ssim_gray,
        'Hist_Corr_R': hist_corr_r,
        'Hist_Corr_G': hist_corr_g,
        'Hist_Corr_B': hist_corr_b,
        'Edge_Similarity': edge_similarity
    }

def identify_abnormality(image1_path, image2_path):
    # Read the images
    img1 = cv2.imread(image1_path)
    img2 = cv2.imread(image2_path)
    
    if img1 is None or img2 is None:
        raise ValueError("One or both images could not be loaded. Check the file paths.")
    
    # Calculate temporal gradients
    grad_rgb, grad_gray = calculate_temporal_gradient(img1, img2)
    
    # Calculate similarity indexes
    similarity_indexes = calculate_similarity_indexes(img1, img2)
    
    # Combine gradients and similarity indexes to identify abnormalities
    # Normalize gradients for visibility
    norm_grad_rgb = cv2.normalize(grad_rgb, None, 0, 255, cv2.NORM_MINMAX)
    norm_grad_gray = cv2.normalize(grad_gray, None, 0, 255, cv2.NORM_MINMAX)
    
    # Create a mask where significant changes or low similarity occur
    threshold = 50  # Set an appropriate threshold value
    abnormal_rgb_mask = np.max(norm_grad_rgb, axis=2) > threshold
    abnormal_gray_mask = norm_grad_gray > threshold

    # Highlight the regions of interest
    highlighted_rgb = img1.copy()
    highlighted_rgb[abnormal_rgb_mask] = [0, 0, 255]  # Mark changes in red
    highlighted_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    highlighted_gray[abnormal_gray_mask] = 255  # Mark changes in white
    
    # Display the results
    cv2.imshow('Original Image 1', img1)
    cv2.imshow('Original Image 2', img2)
    cv2.imshow('Temporal Gradient (RGB)', norm_grad_rgb.astype(np.uint8))
    cv2.imshow('Temporal Gradient (Gray)', norm_grad_gray.astype(np.uint8))
    cv2.imshow('Abnormal Regions (RGB)', highlighted_rgb)
    cv2.imshow('Abnormal Regions (Gray)', highlighted_gray)
    
    # Print similarity indexes
    print(f'Similarity Indexes: {similarity_indexes}')
    
    # Wait until a key is pressed and close the windows
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
#image1_path = 'saved_frames/frame_000.png'
#image2_path = 'saved_frames/frame_001.png'
#identify_abnormality(image1_path, image2_path)


In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.color import rgb2gray

def calculate_temporal_gradient(image1, image2):
    # Convert images to RGB and grayscale
    img1_rgb = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    img1_gray = rgb2gray(img1_rgb)
    img2_gray = rgb2gray(img2_rgb)

    # Calculate the temporal gradient (difference) for each channel and grayscale
    grad_rgb = np.abs(img2_rgb.astype(np.float32) - img1_rgb.astype(np.float32))
    grad_gray = np.abs(img2_gray.astype(np.float32) - img1_gray.astype(np.float32))

    return grad_rgb, grad_gray

def calculate_similarity_indexes(image1, image2):
    # Convert images to RGB and grayscale
    img1_rgb = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    img1_gray = rgb2gray(img1_rgb)
    img2_gray = rgb2gray(img2_rgb)
    
    # Resize images to the same size if they are different
    if img1_rgb.shape != img2_rgb.shape:
        img2_rgb = cv2.resize(img2_rgb, (img1_rgb.shape[1], img1_rgb.shape[0]))
        img2_gray = cv2.resize(img2_gray, (img1_gray.shape[1], img1_gray.shape[0]))

    # Compute Mean Squared Error (MSE) for RGB
    mse_rgb = np.mean((img1_rgb - img2_rgb) ** 2)
    
    # Compute Structural Similarity Index (SSIM) for grayscale
    ssim_gray, _ = ssim(img1_gray, img2_gray, full=True)
    
    # Compute Color Histograms for RGB
    hist1_r = cv2.calcHist([img1_rgb[:, :, 0]], [0], None, [256], [0, 256])
    hist2_r = cv2.calcHist([img2_rgb[:, :, 0]], [0], None, [256], [0, 256])
    hist1_g = cv2.calcHist([img1_rgb[:, :, 1]], [0], None, [256], [0, 256])
    hist2_g = cv2.calcHist([img2_rgb[:, :, 1]], [0], None, [256], [0, 256])
    hist1_b = cv2.calcHist([img1_rgb[:, :, 2]], [0], None, [256], [0, 256])
    hist2_b = cv2.calcHist([img2_rgb[:, :, 2]], [0], None, [256], [0, 256])
    
    hist_corr_r = cv2.compareHist(hist1_r, hist2_r, cv2.HISTCMP_CORREL)
    hist_corr_g = cv2.compareHist(hist1_g, hist2_g, cv2.HISTCMP_CORREL)
    hist_corr_b = cv2.compareHist(hist1_b, hist2_b, cv2.HISTCMP_CORREL)
    
    # Compute Edge-based similarity (using Canny edge detector)
    edges1 = cv2.Canny(cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY), 100, 200)
    edges2 = cv2.Canny(cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY), 100, 200)
    
    edge_similarity = np.sum(edges1 == edges2) / np.size(edges1)
    
    return {
        'MSE_RGB': mse_rgb,
        'SSIM_Gray': ssim_gray,
        'Hist_Corr_R': hist_corr_r,
        'Hist_Corr_G': hist_corr_g,
        'Hist_Corr_B': hist_corr_b,
        'Edge_Similarity': edge_similarity
    }

def process_video(video_path, output_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Error opening video file. Check the file path.")
    
    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Define codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Using MP4 codec
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width * 2, frame_height * 2))  # Updated size for combined frames
    
    ret, prev_frame = cap.read()
    if not ret:
        raise ValueError("Failed to read the first frame.")
    
    prev_frame_rgb = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2RGB)
    prev_frame_gray = rgb2gray(prev_frame_rgb)
    
    while True:
        ret, curr_frame = cap.read()
        if not ret:
            break
        
        curr_frame_rgb = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2RGB)
        curr_frame_gray = rgb2gray(curr_frame_rgb)
        
        # Calculate temporal gradients and similarity indexes
        grad_rgb, grad_gray = calculate_temporal_gradient(prev_frame, curr_frame)
        similarity_indexes = calculate_similarity_indexes(prev_frame, curr_frame)
        
        # Normalize gradients for visibility
        norm_grad_rgb = cv2.normalize(grad_rgb, None, 0, 255, cv2.NORM_MINMAX)
        norm_grad_gray = cv2.normalize(grad_gray, None, 0, 255, cv2.NORM_MINMAX)
        
        # Create masks for abnormalities
        threshold = 50  # Set an appropriate threshold value
        abnormal_rgb_mask = np.max(norm_grad_rgb, axis=2) > threshold
        abnormal_gray_mask = norm_grad_gray > threshold

        # Highlight the regions of interest
        highlighted_rgb = curr_frame.copy()
        highlighted_rgb[abnormal_rgb_mask] = [0, 0, 255]  # Mark changes in red
        highlighted_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
        highlighted_gray[abnormal_gray_mask] = 255  # Mark changes in white
        
        # Combine the results into a single frame
        combined_frame = np.hstack((curr_frame, cv2.cvtColor(highlighted_rgb, cv2.COLOR_BGR2RGB)))
        combined_frame = np.vstack((combined_frame, np.hstack((cv2.cvtColor(highlighted_gray, cv2.COLOR_GRAY2BGR), norm_grad_rgb.astype(np.uint8)))))
        
        # Write the combined frame to the output video
        out.write(combined_frame)
        
        # Update previous frame
        prev_frame_rgb = curr_frame_rgb
        prev_frame_gray = curr_frame_gray
    
    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Example usage
#video_path = 'Cartoonized/U_toon.mp4'
#output_path = 'output_abnormalities.mp4'
#process_video(video_path, output_path)


In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.color import rgb2gray

def calculate_temporal_gradient(image1, image2):
    # Convert images to RGB and grayscale
    img1_rgb = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    img1_gray = rgb2gray(img1_rgb)
    img2_gray = rgb2gray(img2_rgb)

    # Calculate the temporal gradient (difference) for each channel and grayscale
    grad_rgb = np.abs(img2_rgb.astype(np.float32) - img1_rgb.astype(np.float32))
    grad_gray = np.abs(img2_gray.astype(np.float32) - img1_gray.astype(np.float32))

    return grad_rgb, grad_gray

def calculate_similarity_indexes(image1, image2):
    # Convert images to RGB and grayscale
    img1_rgb = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    img1_gray = rgb2gray(img1_rgb)
    img2_gray = rgb2gray(img2_rgb)
    
    # Resize images to the same size if they are different
    if img1_rgb.shape != img2_rgb.shape:
        img2_rgb = cv2.resize(img2_rgb, (img1_rgb.shape[1], img1_rgb.shape[0]))
        img2_gray = cv2.resize(img2_gray, (img1_gray.shape[1], img1_gray.shape[0]))

    # Compute Mean Squared Error (MSE) for RGB
    mse_rgb = np.mean((img1_rgb - img2_rgb) ** 2)
    
    # Compute Structural Similarity Index (SSIM) for grayscale
    ssim_gray, _ = ssim(img1_gray, img2_gray, full=True)
    
    # Compute Color Histograms for RGB
    hist1_r = cv2.calcHist([img1_rgb[:, :, 0]], [0], None, [256], [0, 256])
    hist2_r = cv2.calcHist([img2_rgb[:, :, 0]], [0], None, [256], [0, 256])
    hist1_g = cv2.calcHist([img1_rgb[:, :, 1]], [0], None, [256], [0, 256])
    hist2_g = cv2.calcHist([img2_rgb[:, :, 1]], [0], None, [256], [0, 256])
    hist1_b = cv2.calcHist([img1_rgb[:, :, 2]], [0], None, [256], [0, 256])
    hist2_b = cv2.calcHist([img2_rgb[:, :, 2]], [0], None, [256], [0, 256])
    
    hist_corr_r = cv2.compareHist(hist1_r, hist2_r, cv2.HISTCMP_CORREL)
    hist_corr_g = cv2.compareHist(hist1_g, hist2_g, cv2.HISTCMP_CORREL)
    hist_corr_b = cv2.compareHist(hist1_b, hist2_b, cv2.HISTCMP_CORREL)
    
    # Compute Edge-based similarity (using Canny edge detector)
    edges1 = cv2.Canny(cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY), 100, 200)
    edges2 = cv2.Canny(cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY), 100, 200)
    
    edge_similarity = np.sum(edges1 == edges2) / np.size(edges1)
    
    return {
        'MSE_RGB': mse_rgb,
        'SSIM_Gray': ssim_gray,
        'Hist_Corr_R': hist_corr_r,
        'Hist_Corr_G': hist_corr_g,
        'Hist_Corr_B': hist_corr_b,
        'Edge_Similarity': edge_similarity
    }

def add_label(image, text, position=(10, 30), font_scale=1, color=(255, 255, 255), thickness=2):
    """Add a label to an image."""
    return cv2.putText(image, text, position, cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, thickness)

def process_video(video_path, output_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Error opening video file. Check the file path.")
    
    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Define codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Using MP4 codec
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width * 3, frame_height * 2))  # Updated size for combined frames
    
    ret, prev_frame = cap.read()
    if not ret:
        raise ValueError("Failed to read the first frame.")
    
    prev_frame_rgb = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2RGB)
    prev_frame_gray = rgb2gray(prev_frame_rgb)
    
    while True:
        ret, curr_frame = cap.read()
        if not ret:
            break
        
        curr_frame_rgb = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2RGB)
        curr_frame_gray = rgb2gray(curr_frame_rgb)
        
        # Calculate temporal gradients and similarity indexes
        grad_rgb, grad_gray = calculate_temporal_gradient(prev_frame, curr_frame)
        similarity_indexes = calculate_similarity_indexes(prev_frame, curr_frame)
        
        # Normalize gradients for visibility
        norm_grad_rgb = cv2.normalize(grad_rgb, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        norm_grad_gray = cv2.normalize(grad_gray, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        
        # Convert single-channel grayscale to 3-channel RGB for displaying
        norm_grad_gray_rgb = cv2.cvtColor(norm_grad_gray, cv2.COLOR_GRAY2RGB)
        
        # Create masks for abnormalities
        threshold = 50  # Set an appropriate threshold value
        abnormal_rgb_mask = np.max(norm_grad_rgb, axis=2) > threshold
        abnormal_gray_mask = norm_grad_gray > threshold

        # Highlight the regions of interest
        highlighted_rgb = curr_frame.copy()
        highlighted_rgb[abnormal_rgb_mask] = [0, 0, 255]  # Mark changes in red
        highlighted_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
        highlighted_gray[abnormal_gray_mask] = 255  # Mark changes in white
        
        # Create the combined frame
        combined_frame = np.zeros((frame_height * 2, frame_width * 3, 3), dtype=np.uint8)
        combined_frame[0:frame_height, 0:frame_width] = prev_frame
        combined_frame[0:frame_height, frame_width:frame_width*2] = curr_frame
        combined_frame[0:frame_height, frame_width*2:frame_width*3] = highlighted_rgb
        combined_frame[frame_height:frame_height*2, 0:frame_width] = cv2.cvtColor(highlighted_gray, cv2.COLOR_GRAY2BGR)
        combined_frame[frame_height:frame_height*2, frame_width:frame_width*2] = norm_grad_rgb
        combined_frame[frame_height:frame_height*2, frame_width*2:frame_width*3] = norm_grad_gray_rgb
        
        # Add labels to the combined frame
        combined_frame = add_label(combined_frame, "Previous Frame", position=(10, 30))
        combined_frame = add_label(combined_frame, "Current Frame", position=(frame_width + 10, 30))
        combined_frame = add_label(combined_frame, "Highlighted Changes", position=(frame_width * 2 + 10, 30))
        combined_frame = add_label(combined_frame, "Gray Gradient", position=(10, frame_height + 30))
        combined_frame = add_label(combined_frame, "RGB Gradient", position=(frame_width + 10, frame_height + 30))
        combined_frame = add_label(combined_frame, "Gray Gradient (Norm)", position=(frame_width * 2 + 10, frame_height + 30))
        
        # Write the combined frame to the output video
        out.write(combined_frame)
        
        # Update previous frame
        prev_frame = curr_frame
        prev_frame_rgb = curr_frame_rgb
        prev_frame_gray = curr_frame_gray
    
    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Example usage
#video_path = 'Cartoonized/U_toon.mp4'
#output_path = 'output_abnormalities1.mp4'
#process_video(video_path, output_path)


In [ ]:
import cv2
import numpy as np
from scipy.ndimage import gaussian_filter

def random_smooth_optical_flow_warp(image, max_movement=10, smoothness=50):
    """
    Warp an RGB image using a smooth optical flow field, where flow directions
    are similar across neighboring regions.

    Parameters:
    - image: The input RGB image as a NumPy array.
    - max_movement: Maximum pixel movement allowed for the optical flow vectors. Default is 10.
    - smoothness: Controls how smooth the optical flow is. Higher values make the flow field more uniform.
    
    Returns:
    - warped_image: The warped image based on the smooth optical flow.
    """
    # Get the height and width of the image
    h, w, _ = image.shape

    # Generate random noise-based flow fields for X and Y directions
    flow_X = np.random.uniform(-1, 1, (h, w)).astype(np.float32)
    flow_Y = np.random.uniform(-1, 1, (h, w)).astype(np.float32)

    # Smooth the noise to ensure consistency across regions
    flow_X = gaussian_filter(flow_X, sigma=smoothness)
    flow_Y = gaussian_filter(flow_Y, sigma=smoothness)

    # Normalize the flow fields to the range of [-max_movement, max_movement]
    flow_X = cv2.normalize(flow_X, None, -max_movement, max_movement, cv2.NORM_MINMAX)
    flow_Y = cv2.normalize(flow_Y, None, -max_movement, max_movement, cv2.NORM_MINMAX)

    # Generate a meshgrid of coordinates (X, Y)
    X, Y = np.meshgrid(np.arange(w), np.arange(h))

    # Create a map that distorts the image based on the smoothed optical flow
    map_X = (X + flow_X).astype(np.float32)
    map_Y = (Y + flow_Y).astype(np.float32)

    # Warp the image using remap function
    warped_image = cv2.remap(image, map_X, map_Y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)

    return warped_image

# Example usage
# Load an RGB image
image = cv2.imread('IDB/therock.jpg')

# Apply random smooth optical flow warp
warped_image = random_smooth_optical_flow_warp(image, max_movement=20, smoothness=200)

# Show the result
cv2.imshow("Warped Image", warped_image)
cv2.waitKey(0)
cv2.destroyAllWindows()